In [1]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import sys
sys.path.append('..')
from CurvatureTools import *
from CubicSpline import *
%matplotlib qt

In [2]:
xs = np.array([0, 1.89, 2.88, 3.78, 4.64, 5.47, 6.29, 7.1, 7.92, 8.75, 9.58])
ys = np.arange(11)*10
t = np.linspace(0, 9.58, int(44100*9.58)) # How to sample
position = sample_cubic_spline(xs, ys, t)
velocity = getCurvVectors(position[:, None], 1, 1)[1]*t.size/xs.size
velocityV = velocity.flatten()

#i know the 2nd derivative isnt VMag
#but for sake of testing i kept the name. When refining the code, I will change the name

#VMag = np.sqrt(np.sum(velocity**2, axis=1))
VMag = getCurvVectors(velocity[:, None], 1, 1)[1]

plt.figure(figsize=(6, 12))
plt.subplot(211)
plt.plot(t, position)
plt.xlabel("Time (Sec)")
plt.ylabel("Position (Meters)")
plt.title("Position")

plt.subplot(212)
plt.plot(t, velocityV)
plt.xlabel("Time (Sec)")
plt.ylabel("Velocity (Meters/Sec)")
plt.title("Velocity")

plt.show()

In [3]:
#scale freq range

#setting 1 and 3 harmonies
positionSO = position[:] * 100 * (2**(0/12))
posSO = np.sin(2*np.pi*positionSO)
positionSTH = position[:] * 100 * (2**(7/12))
posSTH = np.sin(2*np.pi*positionSTH)

#creating array for 2,4,5 harmonies 
positionST = np.linspace(0, 9.58, int(44100*9.58))
positionSF = np.linspace(0, 9.58, int(44100*9.58))
positionFN = np.linspace(0, 9.58, int(44100*9.58))

for i in range(len(position)):
    positionFN[i] = 0
    if i == len(position)-1:
        #do major 9
        #print("did major 9")
        positionST[i] = position[i] * 100 * (2**(4/12))
        positionSF[i] = position[i] * 100 * (2**(11/12))
        positionFN[i] = position[i] * 100 * (2**(14/12))
    elif VMag[i+1] < VMag[i]:
        #do minor 7th
        #print("did minor 7")
        positionST[i] = position[i] * 100 * (2**(3/12))
        positionSF[i] = position[i] * 100 * (2**(10/12))
    else:
        #do major 7th
        #print("did major 7")
        positionST[i] = position[i] * 100 * (2**(4/12))
        positionSF[i] = position[i] * 100 * (2**(11/12))

#making 2,4,5 sin waves
posST = np.sin(2*np.pi*positionST)    
posSF = np.sin(2*np.pi*positionSF)  
posFN = np.sin(2*np.pi*positionFN)
  
#creating chord
posaudio = posSO + posST + posSTH + posSF + posFN 

In [4]:
ipd.Audio(posaudio, rate = 44100)